### Digits dataset

In [4]:
import numpy as np
import sklearn
from sklearn import datasets
try:
    import umap
except ImportError:
    print("This example requires the UMAP library. You can install it with the command `!pip install umap-learn`")
import warnings
warnings.filterwarnings("ignore")

In [5]:
import kmapper as km
from kmapper.plotlyviz import *
import plotly.graph_objs as go
import ipywidgets as ipw

In [6]:
data, labels = datasets.load_digits().data, datasets.load_digits().target

In [7]:
mapper = km.KeplerMapper(verbose=0)

projected_data = mapper.fit_transform(data, projection=umap.UMAP(n_neighbors=8,
                                                                 min_dist=0.65,
                                                                 n_components=2,
                                                                 metric='euclidean',
                                                                 random_state=3571))

# Get the simplicial complex  
scomplex = mapper.map(projected_data,
                      clusterer=sklearn.cluster.DBSCAN(eps=0.3, min_samples=15),
                      coverer=km.Cover(35, 0.9))

The mapper graph nodes associated to this data set are colored according to the `color_function` values with Viridis colorscale,
which is the default color scale, defined in `plotlyviz`.

In [ ]:
color_function = projected_data[:, 1]-projected_data[:, 1].min()
kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, color_function=color_function,
                                                                 color_function_name='Distance to y-min')
for node in kmgraph['nodes']:
    node['custom_tooltips'] = labels[np.array(node['member_ids'])]
graph_info = get_kmgraph_meta(mapper_summary)

In [ ]:
#PLot the graph with Kamada Kawai layout
plgraph_data = plotly_graph(kmgraph, graph_layout='fr', factor_size=3, edge_linewidth=1, 
                            node_linecolor='rgb(100,100,100)')
layout = plot_layout(title='Mapper graph of digits dataset', width=620, height=620,
                     annotation_text=graph_info, bgcolor='rgb(240,240,240)'  
                      )

fw_graph = go.FigureWidget(data=plgraph_data, layout=layout) # mapper graph figure

The generated FigureWidget can be updated (by performing a restyle or relayout). For example, let us add a title 
to the colorbar (the name of the color function, if any),
and set the title font size. To perform these updates faster, Plotly 3.+ provides a context manager that batches up all data and
layout updates:

In [ ]:
with fw_graph.batch_update():
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>y-min'
    fw_graph.data[1].marker.colorbar.titlefont.size = 10

In [ ]:
fw_hist = node_hist_fig(colorf_distribution,  left=25, right=10) # default  width=400, height=300,
fw_summary = summary_fig(mapper_summary, width=600, height=350, left=10, right=20) #default width=600 

In [ ]:
ipw.VBox([fw_graph, ipw.HBox([ fw_summary, fw_hist])])

In [ ]:
dashboard = hovering_widgets(kmgraph, fw_graph, ctooltips=True, member_textbox_width=500)

In [ ]:
dashboard

To save any of the FigureWidget(s) uncomment the lines in the next cell:

In [ ]:
#import plotly.io as pio
#pio.write_image(fw_graph, 'digits.pdf')#or png, swg

The dasboards, i.e. the boxes containing two or more FigureWidget(s) cannot be saved in a single image file. First save its
FigureWidget(s) components, and then arrange them in the same order in a LaTeX file, for example.